In [1]:
!pip install transformers

In [2]:
!pip install sentencepiece

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os

In [4]:
# tokenization_kobert.py 업로드
from google.colab import files
files.upload()

{}

In [5]:
# kobert tokenize import
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [6]:
# train, validation, test파일 업로드

from google.colab import files
uploaded = files.upload()

In [7]:
train = pd.read_table('final_train_1.txt', names = ['label', 'comment'])
validation = pd.read_table('final_validation_1.txt', names = ['label', 'comment'])
test = pd.read_table('final_test_1.txt', names = ['label', 'comment'])
train.head()

,label,comment
0,0,바가지를너무씌워서다시는안가기로했읍니다.지금이어느땐데가격표도없구부르는데로받더라구요.한...
1,0,날씨와 코로나때문에 아쉬웠습니다. 놀이기구 타는 시간도 너무 짧아서 타다만느낌이었어요ㅜㅜ
2,0,망원한강공원가는 출입구가 현재는 하나인데 그마저도 너무 협소 사람 둘이 서서 간신히...
3,0,탈수있는게 별로 없어요
4,0,매표소 직원분들이 온라인 예매에 대해 잘 모르시는 것 같습니다 당일온라인예매는 당일...


In [8]:
def convert_data(data_df):
  global tokenizer

  SEQ_LEN = 250 # 버트 인풋 길이

  tokens, masks, segments, targets = [], [], [], []

  for i in tqdm(range(len(data_df))):
    # 토큰 인풋
    token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length = SEQ_LEN, truncation = True, padding = 'max_length')

    # 마스크 인풋
    num_zeros = token.count(0)
    mask = [1] * (SEQ_LEN - num_zeros) + [0] * num_zeros 

    # 세그먼트 인풋
    segment = [0] * SEQ_LEN

    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

    targets.append(data_df[LABEL_COLUMN][i])

  # 배열로 바꾸기
  tokens = np.array(tokens)
  masks = np.array(masks)
  segments = np.array(segments)
  targets = np.array(targets)

  return [tokens, masks, segments], targets

# convert_data를 불러오는 함수
def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
  data_x, data_y = convert_data(data_df)
  return data_x, data_y

SEQ_LEN = 250
BATCH_SIZE = 20

DATA_COLUMN = 'comment'
LABEL_COLUMN = 'label'

train_x, train_y = load_data(train)
validation_x, validation_y = load_data(validation)
test_x, test_y = load_data(test)

100%|██████████| 1800/1800 [00:00<00:00, 3951.95it/s]


In [9]:
from tensorflow.keras.utils import to_categorical

train_y = to_categorical(train_y)
validation_y = to_categorical(validation_y)
test_y = to_categorical(test_y)

In [10]:
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.57.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.57.186:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [19]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate = 5.0e-5, decay = 0.0025)

In [20]:
# 감성 분석에 맞는 버트 모형을 리턴하는 함수
def create_sentiment_bert():
  model = TFBertModel.from_pretrained('monologg/kobert', from_pt=True)
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
  sentiment_first = tf.keras.layers.Dense(3, activation = 'softmax', 
                                          kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))(sentiment_drop)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
  sentiment_model.compile(optimizer = opt, 
                          loss = tf.keras.losses.CategoricalCrossentropy(),
                          metrics = ['accuracy'])
  return sentiment_model

In [ ]:
# 버트를 활용한 감성분석 모델 만들기
# TPU 객체 지정
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass
  
# TPU 실행 시
if TPU:
  strategy = tf.distribute.TPUStrategy(resolver)
  with strategy.scope():
    sentiment_model = create_sentiment_bert()
  sentiment_model.fit(train_x, train_y,
                      epochs = 20, shuffle = True, batch_size = 128,
                      validation_data = (validation_x, validation_y))
else: # GPU
  sentiment_model = create_sentiment_bert()
  sentiment_model.fit(train_x, train_y,
                      epochs = 10, shuffle = True, batch_size = 128,
                      validation_data = (validation_x, validation_y))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.57.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.57.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

Epoch 1/20


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 3) dtype=float32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 3) dtype=float32>]


66/66 [==============================] - ETA: 0s - loss: 1.7719 - accuracy: 0.3371

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 250) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None, 3) dtype=float32>]


66/66 [==============================] - 172s 1s/step - loss: 1.7719 - accuracy: 0.3371 - val_loss: 1.1005 - val_accuracy: 0.3333
Epoch 2/20
66/66 [==============================] - 17s 256ms/step - loss: 1.2851 - accuracy: 0.3252 - val_loss: 1.1053 - val_accuracy: 0.3333
Epoch 3/20
66/66 [==============================] - 17s 256ms/step - loss: 1.2057 - accuracy: 0.3296 - val_loss: 1.2447 - val_accuracy: 0.3333
Epoch 4/20
66/66 [==============================] - 17s 256ms/step - loss: 1.2263 - accuracy: 0.3302 - val_loss: 1.2221 - val_accuracy: 0.3333
Epoch 5/20
66/66 [==============================] - 17s 256ms/step - loss: 1.2491 - accuracy: 0.3327 - val_loss: 1.1424 - val_accuracy: 0.3333
Epoch 6/20
66/66 [==============================] - 17s 257ms/step - loss: 1.1794 - accuracy: 0.3323 - val_loss: 1.0999 - val_accuracy: 0.3333
Epoch 7/20
66/66 [==============================] - 17s 257ms/step - loss: 1.1921 - accuracy: 0.3394 - val_loss: 1.1392 - val_accuracy: 0.3333
Epoch 8/20
6

In [ ]:
from sklearn.metrics import accuracy_score

preds = sentiment_model.predict(test_x)
accuracy_score(test_y, np.round(preds, 0))